In [1]:
import sqlite3
import pandas as pd
import numpy as np
import time
import pickle
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize.moses import MosesDetokenizer

from stop_words import get_stop_words
from collections import defaultdict
import gensim
from gensim import corpora, models, similarities
from gensim.models.doc2vec import TaggedDocument,TaggedLineDocument
from gensim.models import Doc2Vec
import gensim.models.doc2vec

## Full Body Model Training

In [2]:
conn = sqlite3.connect("volume2/testDB.db")

In [3]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('PLOS_ALL',), ('PLOS_ALL_tok',), ('ed_concat',)]


In [4]:
df0 = pd.read_sql('SELECT * from PLOS_ALL_tok LIMIT 10', conn)
df0.head()

,index,doi,bod_toks,ab_toks
0,0,10.1371/journal.pone.0143437,"['pulmonari', 'prostat', 'cancer', 'common', '...","['bone', 'scintigraphi', 'wide', 'appli', 'det..."
1,1,10.1371/journal.pone.0096769,"['voic', 'disord', 'dysphonia', 'can', 'caus',...","['cystic', 'fibrosi', 'common', 'autosom', 're..."
2,2,10.1371/journal.pone.0177044,"['dementia', 'typic', 'defin', 'clinic', 'synd...","['alzheim', 'diseas', 'neurodegen', 'syndrom',..."
3,3,10.1371/journal.pone.0033200,"['non', 'small', 'cell', 'lung', 'cancer', 'ns...","['purposexeroderma', 'pigmentosum', 'group', '..."
4,4,10.1371/journal.pone.0001814,"['endosom', 'rout', 'consid', 'default', 'path...","['class', 'mhc', 'molecul', 'display', 'peptid..."


In [6]:
df = pd.read_sql('SELECT * from PLOS_ALL LIMIT 10', conn)
df.head()

,level_0,index,doi,pmid,journal,publisher_id,publisher_name,authors,editors,subjects,title,alt_title,date_pub,date_rec,abstract,body
0,0,0,10.1371/journal.pone.0143437,26599401,PLoS ONE,plos,Public Library of Science,"['Changyin Wang', 'Ying Shen', 'Shaobo Zhu']",['Harriet Wikman'],['Research Article'],Distribution Features of Skeletal Metastases: ...,Distribution Features of Skeletal Metastases,2015-11-23,2015-6-24,Bone scintigraphies are widely applied for det...,Pulmonary and prostate cancers are the most co...
1,1,1,10.1371/journal.pone.0096769,24796691,PLoS ONE,plos,Public Library of Science,"['Bruna Mendes Lourenço', 'Kauê Machado Costa'...",['Donald A. Robin'],"['Research Article', 'Biology and Life Science...",Voice Disorder in Cystic Fibrosis Patients,Voice Disorder in Cystic Fibrosis,2014-5-5,2013-10-10,Cystic fibrosis common autosomal recessive d...,"Voice disorders, or dysphonias, can be caused ..."
2,2,2,10.1371/journal.pone.0177044,28472200,PLoS ONE,plos,Public Library of Science,"['Miriam L. Haaksma', 'Lara R. Vilela', 'Aless...",['Stephen D. Ginsberg'],"['Research Article', 'Medicine and Health Scie...",Comorbidity and progression of late onset Alzh...,Comorbidity and progression of late onset Alzh...,2017-5-4,2017-2-2,Alzheimer’ disease neurodegenerative syndrom...,1. Dementia is typically defined as a clinical...
3,3,3,10.1371/journal.pone.0033200,22479369,PLoS ONE,plos,Public Library of Science,"['Wenting Wu', 'Huan Li', 'Huibo Wang', 'Xueyi...",['Alfons Navarro'],"['Research Article', 'Biology', 'Genetics', 'H...",Effect of Polymorphisms in,None,2012-3-29,2011-11-9,PurposeXeroderma pigmentosum group (XPD) code...,Non-small cell lung cancer (NSCLC) accounts fo...
4,4,4,10.1371/journal.pone.0001814,18350151,PLoS ONE,plos,Public Library of Science,"['Shashank Gupta', 'Sabine Höpner', 'Bernd Rup...",['Mauricio Martins Rodrigues'],"['Research Article', 'Immunology/Antigen Proce...",Anchor Side Chains of Short Peptide Fragments ...,Ligand-Exchange Catalysts,2008-3-19,2008-1-10,Class MHC molecules display peptides the cel...,The endosomal route is considered to be the de...


In [7]:
docs = [doc for doc in TaggedLineDocument('volume2/processed_body_docs.txt')]

In [ ]:
docs[200000]

In [4]:
df = pd.read_pickle('volume/bodtokens_0.pkl')

In [5]:
df.head()

,doi,bod_toks
0,10.1371/journal.pone.0143437,"[pulmonari, prostat, cancer, common, malign, t..."
1,10.1371/journal.pone.0096769,"[voic, disord, dysphonia, can, caus, neurolog,..."
2,10.1371/journal.pone.0177044,"[dementia, typic, defin, clinic, syndrom, cogn..."
3,10.1371/journal.pone.0033200,"[non, small, cell, lung, cancer, nsclc, accoun..."
4,10.1371/journal.pone.0001814,"[endosom, rout, consid, default, pathway, load..."


In [16]:
df0 = pd.read_sql('SELECT * from PLOS_ALL_tok', conn)
df0.head()

,level_0,index,doi,bod_toks
0,0,0,10.1371/journal.pone.0143437,"['pulmonari', 'prostat', 'cancer', 'common', '..."
1,1,1,10.1371/journal.pone.0096769,"['voic', 'disord', 'dysphonia', 'can', 'caus',..."
2,2,2,10.1371/journal.pone.0177044,"['dementia', 'typic', 'defin', 'clinic', 'synd..."
3,3,3,10.1371/journal.pone.0033200,"['non', 'small', 'cell', 'lung', 'cancer', 'ns..."
4,4,4,10.1371/journal.pone.0001814,"['endosom', 'rout', 'consid', 'default', 'path..."


In [ ]:
### TRAINING -- ONCE YOU READ IN FROM DB (STR)

In [16]:
tdf.head()

,level_0,index,doi,bod_toks
0,0,0,10.1371/journal.pone.0143437,"['pulmonari', 'prostat', 'cancer', 'common', '..."
1,1,1,10.1371/journal.pone.0096769,"['voic', 'disord', 'dysphonia', 'can', 'caus',..."
2,2,2,10.1371/journal.pone.0177044,"['dementia', 'typic', 'defin', 'clinic', 'synd..."
3,3,3,10.1371/journal.pone.0033200,"['non', 'small', 'cell', 'lung', 'cancer', 'ns..."
4,4,4,10.1371/journal.pone.0001814,"['endosom', 'rout', 'consid', 'default', 'path..."


In [17]:
bodtexts = tdf['bod_toks'].tolist()

In [32]:
tok = bodtexts[1].split(', ')

In [40]:
tnew = []
for line in tok:
    l = line.replace("'","")
    l = l.strip('[]')
    tnew.append(l)

In [18]:
bods = []

for i in range(0,len(bodtexts)):
    det = bodtexts[i].replace(',','')
    det = det.replace("'","")
    det = det.strip('[]')
    bods.append(det)
    if i % 20000 == 0:
        print(i)

0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000


In [ ]:
bods[1]

#### toklen

In [ ]:
tok_len = []

for i in range(0,len(bods)):
    tok_len.append(len(bods[i].split()))
    if i % 10000 == 0:
        print(i)
df = pd.DataFrame(tok_len)
df.describe()

In [32]:
df.columns = ['toklen']

In [34]:
df[df['toklen'] > 10000].count() 

toklen    449
dtype: int64

In [35]:
df[df['toklen'] > 10000].describe()

,toklen
count,449.000000
mean,11786.195991
std,2063.130425
min,10001.000000
25%,10523.000000
50%,11114.000000
75%,12338.000000
max,27798.000000


In [26]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [29]:
corpora.MmCorpus.serialize('/volume/models/corpus.mm', corpus)
dictionary.save('/volume/models/dictionary.dict')

In [30]:
tfidf = models.TfidfModel(corpus, normalize=True)
tfidf.save('/volume/model.tfidf')

In [31]:
start_time = time.time()
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=200)
lsi.save('/volume/models/model.lsi')
print("--- %s seconds ---" % (time.time() - start_time))
#corpus_lsi = lsi[corpus_tfidf]

In [32]:
start_time = time.time()
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=200)
lda.save('/volume/models/model.lda')
print("--- %s seconds ---" % (time.time() - start_time))
#corpus_lda = lda[corpus_tfidf]

In [ ]:
corpus_tfidf = tfidf[corpus]
corpus_lsi = lsi[corpus_tfidf]
corpus_lda = lda[corpus_tfidf]

In [10]:
index = similarities.MatrixSimilarity(lsi[corpus])
index.save('/volume/models/similarity.index')

In [9]:
file = open('/volume/processed_body_docs.txt', 'w')

for i in range(0,len(bods)):
    detok_str = bods[i]
    file.write("%s\n" % detok_str)

In [3]:
documents = []
with open('/volume/processed_body_docs.txt') as f:
    for line in f:
        documents.append(TaggedLineDocument(line))

In [9]:
type(documents[1])

gensim.models.doc2vec.TaggedLineDocument

In [5]:
# detok for abstracts 

detokenizer = MosesDetokenizer()
file = open('/volume/processed_docs.txt', 'w')

for i in range(0,len(texts)):
    detok_str = detokenizer.detokenize(texts[i], return_str=True)
    file.write("%s\n" % detok_str)

In [26]:
bods_sm = bods[0:10]

In [27]:
len(bods_sm)

10

In [ ]:
#documents = [doc for doc in TaggedLineDocument('/volume/processed_docs.txt')]
documents = [doc for doc in TaggedLineDocument(bods_sm)]


In [ ]:
model = Doc2Vec(documents, size=200, window=5, min_count=5,workers=15, iter=10)

np.save('models/body_features-w2v-200.npy',model.docvecs.doctag_syn0)
model.save('models/body_features-w2v-200.doc2vec')